In [2]:
#importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler

DATASET

The data set was generated by a Monte Carlo program, Corsika, described in:
    D. Heck et al., CORSIKA, A Monte Carlo code to simulate extensive air showers,
    Forschungszentrum Karlsruhe FZKA 6019 (1998).
http://rexa.info/paper?id=ac6e674e9af20979b23d3ed4521f1570765e8d68

In [3]:
#column names
cols = ['fLength', 'fWidth', 'fSize', 'fConc', 'fConc1', 'fAsym', 'fM3Long', 'fM3Trans', 'fAlpha', 'fDist', 'class'];

#reading the data from the directory and storing it in a dataframe with the column names;
df = pd.read_csv('./gamma_data/magic04.data', names=cols);
#df.head();
print(df.head())

    fLength    fWidth   fSize   fConc  fConc1     fAsym  fM3Long  fM3Trans  \
0   28.7967   16.0021  2.6449  0.3918  0.1982   27.7004  22.0110   -8.2027   
1   31.6036   11.7235  2.5185  0.5303  0.3773   26.2722  23.8238   -9.9574   
2  162.0520  136.0310  4.0612  0.0374  0.0187  116.7410 -64.8580  -45.2160   
3   23.8172    9.5728  2.3385  0.6147  0.3922   27.2107  -6.4633   -7.1513   
4   75.1362   30.9205  3.1611  0.3168  0.1832   -5.5277  28.5525   21.8393   

    fAlpha     fDist class  
0  40.0920   81.8828     g  
1   6.3609  205.2610     g  
2  76.9600  256.7880     g  
3  10.4490  116.7370     g  
4   4.6480  356.4620     g  


In [4]:
#converting the class labels to 0 and 1 because computer do not understand letters;
df['class'] = df['class'].map({'g':0, 'h':1});
print(df.head());

    fLength    fWidth   fSize   fConc  fConc1     fAsym  fM3Long  fM3Trans  \
0   28.7967   16.0021  2.6449  0.3918  0.1982   27.7004  22.0110   -8.2027   
1   31.6036   11.7235  2.5185  0.5303  0.3773   26.2722  23.8238   -9.9574   
2  162.0520  136.0310  4.0612  0.0374  0.0187  116.7410 -64.8580  -45.2160   
3   23.8172    9.5728  2.3385  0.6147  0.3922   27.2107  -6.4633   -7.1513   
4   75.1362   30.9205  3.1611  0.3168  0.1832   -5.5277  28.5525   21.8393   

    fAlpha     fDist  class  
0  40.0920   81.8828      0  
1   6.3609  205.2610      0  
2  76.9600  256.7880      0  
3  10.4490  116.7370      0  
4   4.6480  356.4620      0  


In [ ]:
#looping through the columns and plotting the histogram for each column;
for label in cols[:-1]:
  plt.hist(df[df["class"]==0][label], color='blue', label='gamma', alpha=0.7, density=True)
  plt.hist(df[df["class"]==1][label], color='red', label='hadron', alpha=0.7, density=True)
  plt.title(label)
  plt.ylabel("Probability")
  plt.xlabel(label)
  plt.legend()
  #plt.show()

Train , Validation and Test

In [11]:
#splitting the data into training, validation and testing data;
#The first 60% of the data goes to train (training set).
#The next 20% goes to valid (validation set).
#The remaining 20% goes to test (testing set).
train, valid, test = np.split(df.sample(frac=1), [int(0.6*len(df)), int(0.8*len(df))])

In [7]:
#scaling the data;
#The data is scaled to have a mean of 0 and a standard deviation of 1.
#Machine learning models perform better with scaled data.
#Prevents models from being biased toward large-valued features.

def scale_dataset(dataframe,  oversample=False):
  X = dataframe[dataframe.columns[:-1]].values
  y = dataframe[dataframe.columns[-1]].values

  scaler = StandardScaler()
  X = scaler.fit_transform(X)

  if oversample:
    ros = RandomOverSampler()
    X, y = ros.fit_resample(X, y)

  data = np.hstack((X, np.reshape(y, (-1, 1))))

  return data, X, y

In [ ]:
#scaling the training, validation and testing data;
train, X_train, y_train = scale_dataset(train, oversample=True)
valid, X_valid, y_valid = scale_dataset(valid, oversample=False)
test, X_test, y_test = scale_dataset(test, oversample=False)

K-NN [ K-Nearest Neighbors ]

A machine learning algorithm used for classification and regression.
Examples:
Imagine you want to classify a fruit as an apple or an orange based on its color and size.

You plot existing apples and oranges on a graph.
A new fruit appears, and you check its K nearest neighbors.
If most neighbors are apples, classify it as an apple. If most are oranges, classify it as an orange.

Key Points:
✔ Simple & Easy to implement
✔ Works well with small datasets
❌ Slow for large datasets (since it stores all data and searches through it)
❌ Sensitive to noise (bad data can affect results)

In [12]:
from sklearn.neighbors import KNeighborsClassifier #importing the KNN classifier
from sklearn.metrics import classification_report  #importing the classification report

In [ ]:
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train, y_train)

In [14]:
y_pred = knn_model.predict(X_test)

In [15]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.87      0.86      2450
           1       0.75      0.72      0.73      1354

    accuracy                           0.81      3804
   macro avg       0.80      0.79      0.80      3804
weighted avg       0.81      0.81      0.81      3804



In [ ]:
#stop at 1hr : 11